В рамках выполнения данного задания вам необходимо создать панораму вашей комнаты. Панорама должна состоять минимум из 5 изображений. Они должны быть сняты таким образом, чтобы любое из них имело пересечение не более, чем с двумя другими

Баллы будут проставлены следующим образом:
* До 7 баллов за построение панорамы (зависит от её итогового качества)
* 3 балла за самостоятельную имплементацию любого из рассмотренных на лекции или семинаре алгоритма

In [ ]:
%pip install opencv-python matplotlib numpy ipython-autotime

In [ ]:
import glob
import itertools
import math
import os.path as osp
import time

import cv2
import matplotlib
import numpy as np

matplotlib.use('TkAgg')  # for macOS

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 16.0)
plt.rcParams['image.interpolation'] = 'bilinear'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2
%load_ext autotime

# Panorama stitching

## Load Images

In [ ]:
input_dir = 'input'
ipaths = sorted(glob.glob(osp.join(input_dir, 'img_*.jpg')))
n_imgs = len(ipaths)

images_info = dict()
for i_num, ipath in enumerate(ipaths):
    img = plt.imread(ipath)
    images_info[ipath] = {'img': img}
    plt.subplot(1, n_imgs, i_num + 1)
    plt.title(osp.basename(ipath), fontsize=20)
    plt.axis('off')
    plt.imshow(images_info[ipath]['img'])
plt.show()

## Keypoints detection + descriptors

In [ ]:
def keypoints_detection_sift(input_img):
    gray = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT_create()
    kps, dscrs = sift.detectAndCompute(gray, None)
    return kps, dscrs

In [ ]:
for i_num, ipath in enumerate(images_info):
    img = images_info[ipath]['img']
    keypoints, descriptors = keypoints_detection_sift(img)
    images_info[ipath]['keypoints'] = keypoints
    images_info[ipath]['descriptors'] = descriptors
    plt.subplot(1, n_imgs, i_num + 1)
    plt.title(osp.basename(ipaths[i_num]), fontsize=20)
    plt.imshow(cv2.drawKeypoints(
        img, keypoints, None,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    ))
    plt.axis('off')
plt.show()

In [ ]:
keypoints, descriptors = images_info[ipaths[0]]['keypoints'], \
    images_info[ipaths[0]]['descriptors']
keypoint = sorted(keypoints, key=lambda x: x.size, reverse=True)[0]
for field in dir(keypoint):
    if not field.startswith('_'):
        print(f'{field:10} {getattr(keypoint, field)}')

In [ ]:
_ = plt.figure(figsize=(10, 8))
plt.imshow(cv2.drawKeypoints(
    images_info[ipaths[0]]['img'], [keypoint, ], None,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
)
plt.axis('off')
plt.show()

## Keypoints matching

### Bruteforce + Cross-check

In [ ]:
def keypoints_matching_cross_check(dscrs1, dscrs2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    cross_matches = bf.match(dscrs1, dscrs2)
    return cross_matches

In [ ]:
matches = keypoints_matching_cross_check(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

### KNN + Ratio test

In [ ]:
def keypoints_matching_knn(dscrs1, dscrs2):
    bf = cv2.BFMatcher()
    tic = time.time_ns()
    knn_matches = bf.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    print(f'Matching time: {(toc - tic) / 10e6 :.5f} ms')
    good_matches = []
    for neighbour_1, neighbour_2 in knn_matches:
        if neighbour_1.distance < 0.75 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

In [ ]:
matches = keypoints_matching_knn(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

### FLANN

In [ ]:
def keypoints_matching_flann(dscrs1, dscrs2):
    index_params = dict(
        algorithm=1,  # FLANN_INDEX_KDTREE
        trees=5
    )
    search_params = dict(checks=20)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    tic = time.time_ns()
    flann_matches = flann.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    print(f'Matching time: {(toc - tic) / 10e6 :.5f} ms')
    good_matches = []
    for neighbour_1, neighbour_2 in flann_matches:
        if neighbour_1.distance < 0.75 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

In [ ]:
matches = keypoints_matching_flann(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

In [ ]:
print(f'Descriptors shape on the image_0: {images_info[ipaths[0]]["descriptors"].shape}')
print(f'Descriptors shape on the image_1: {images_info[ipaths[1]]["descriptors"].shape}')
mock_descriptors_1 = np.repeat(images_info[ipaths[0]]['descriptors'], 100, axis=0)
mock_descriptors_2 = np.repeat(images_info[ipaths[1]]['descriptors'], 100, axis=0)
print(f'Descriptors shape on the mock_image_0: {mock_descriptors_1.shape}')
print(f'Descriptors shape on the mock_image_1: {mock_descriptors_2.shape}')

In [ ]:
_ = keypoints_matching_knn(mock_descriptors_1, mock_descriptors_2)

In [ ]:
_ = keypoints_matching_flann(mock_descriptors_1, mock_descriptors_2)

### Find matches for images

In [ ]:
matches_images = dict()
for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
    matches_pair = keypoints_matching_knn(
        images_info[ipath_0]['descriptors'],
        images_info[ipath_1]['descriptors']
    )
    matches_images[(ipath_0, ipath_1)] = matches_pair

## Homography

In [ ]:
homographies = dict()
for (ipath_0, ipath_1), matches in matches_images.items():
    src_pts = np.float32(
        [images_info[ipath_0]['keypoints'][m.queryIdx].pt for m in matches]
    ).reshape(-1, 1, 2)
    dst_pts = np.float32(
        [images_info[ipath_1]['keypoints'][m.trainIdx].pt for m in matches]
    ).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(
        src_pts,
        dst_pts,
        cv2.RANSAC,
        ransacReprojThreshold=.5,
        maxIters=20000,
        confidence=0.995
    )
    homographies[ipath_0, ipath_1] = (M, mask.ravel().tolist())

In [ ]:
for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
    matches = matches_images[ipath_0, ipath_1]
    M, matches_mask = homographies[ipath_0, ipath_1]
    img_0, img_1_polylines = images_info[ipath_0]['img'].copy(), \
        images_info[ipath_1]['img'].copy()
    rows_0, cols_0 = img_0.shape[:2]
    rows_1, cols_1 = img_1_polylines.shape[:2]
    pts = np.float32(
        [[0, 0], [0, rows_0], [cols_0, rows_0], [cols_0, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    img_1_polylines = cv2.polylines(
        img_1_polylines, [np.int32(dst)], True, 255, 3, cv2.LINE_AA
    )
    img_matches = cv2.drawMatches(
        img_0,
        images_info[ipath_0]['keypoints'],
        img_1_polylines,
        images_info[ipath_1]['keypoints'],
        matches,
        None,  # output image
        matchesMask=matches_mask,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    plt.title(
        f'Pair of images: {ipath_0} and {ipath_1} ({sum(matches_mask)} matches)',
        fontsize=20
    )
    plt.imshow(img_matches)
    plt.axis('off')
    plt.show()

## Stitching images

In [ ]:
M, _ = homographies[ipaths[1], ipaths[0]]
img_0, img_1 = images_info[ipaths[0]]['img'], images_info[ipaths[1]]['img']
img_0_1 = cv2.warpPerspective(
    img_1, M, (img_0.shape[1] + img_1.shape[1], img_1.shape[0])
)
img_0_1[0:img_0.shape[0], 0:img_0.shape[1]] = img_0

# Some postprocessing
img_0_1_mask = np.where(img_0_1.sum(axis=2) != 0, 1, 0).astype('uint8')
kernel = np.ones((5, 5), np.uint8)
img_0_1_mask = cv2.morphologyEx(img_0_1_mask, cv2.MORPH_CLOSE, kernel)
img_0_1_mask = cv2.erode(img_0_1_mask, kernel, iterations=1)
img_0_1_mask = np.dstack([img_0_1_mask, ] * 3)

In [ ]:
plt.imshow(img_0_1)
plt.axis('off')
plt.show()

In [ ]:
M, _ = homographies[ipaths[2], ipaths[0]]
img_2 = images_info[ipaths[2]]['img']
img_2_pano = cv2.warpPerspective(
    img_2, M, (img_0.shape[1] + img_1.shape[1], img_1.shape[0])
)
panorama = np.where(img_0_1_mask != 0, img_0_1, img_2_pano)

In [ ]:
plt.imshow(panorama)
plt.show()

# Camera calibration

Для калибровки камер мы будем использовать специальный паттерн, который называется шахматная доска. Его параметры нам хорошо известны, поэтому с его помощью откалибровать камеру будет проще, чем через пайплайн, который мы использовали для построения панорамы.
Изображение chessboard можно найти в поставке opencv на [github](https://github.com/opencv/opencv/blob/4.x/samples/data/chessboard.png)
Его необходимо распечатать и сфотографировать с разных углов и расстояний, чтобы получить достаточное количество изображений для калибровки камеры. Суммарное количество изображений должно быть не менее 10.

In [ ]:
input_dir = 'input'
chessboard_ipaths = sorted(glob.glob(osp.join(input_dir, 'chessboard_*.jpg')))
n_chessboard_imgs = len(chessboard_ipaths)
print(f'Number of chessboard images: {n_chessboard_imgs}')

На каждом изображении необходимо найти углы шахматной доски. Для этого используется функция `cv2.findChessboardCorners`. С помощью этой функции мы получим соответствие между углами шахматной доски в реальном мире (object points) и точками на изображении (image points)
После нахождения углов их местоположение можно доуточнить с помощью функции `cv2.cornerSubPix`. Вы можете воспользоваться этой функцией, если вам будет недостаточно получаемого без неё качества калибровки.

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((6 * 9, 3), np.float32)
objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images
obj_points = []  # 3d points in real world space
img_points = []  # 2d points in image plane.

In [ ]:
_ = plt.figure(figsize=(16, 36))
for i_num, ipath in enumerate(chessboard_ipaths):
    img = cv2.cvtColor(cv2.imread(ipath), cv2.COLOR_BGR2RGB)
    img_grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    result, corners = cv2.findChessboardCorners(img_grayscale, (9, 6), None)
    if result:
        obj_points.append(objp)
        img_points.append(corners)

        cv2.drawChessboardCorners(img, (9, 6), corners, result)
        plt.subplot(math.ceil(n_chessboard_imgs / 3), 3, i_num + 1)
        plt.title(osp.basename(ipath), fontsize=12)
        plt.axis('off')
        plt.imshow(img)
plt.show()

Имея соответствие между углами шахматной доски в реальном мире и точками на изображении, можно калибровать камеру. Для этого используется функция `cv2.calibrateCamera`. В результате калибровки мы получим матрицу камеры и коэффициенты дисторсии.

In [ ]:
img = cv2.cvtColor(cv2.imread(chessboard_ipaths[-1]), cv2.COLOR_BGR2RGB)
img_grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
result, mtx, dist, r_vecs, t_vecs = cv2.calibrateCamera(
    objectPoints=obj_points,
    imagePoints=img_points,
    imageSize=img_grayscale.shape[::-1],
    cameraMatrix=None,
    distCoeffs=None
)
print(f'Camera matrix:\n{mtx}')
print(f'Distortion coefficients:\n{dist}')

In [ ]:
np.savetxt('mtx.txt', mtx)
np.savetxt('dist.txt', dist)

Эти параметры можно использовать для коррекции изображений, полученных с данной камеры. Для этого используется функция `cv2.undistort`. При этом можно использовать функцию `cv2.getOptimalNewCameraMatrix`, чтобы получить новую матрицу камеры. Она будет содержать только те пиксели, которые содержат информацию об изображении.

In [ ]:
rows, cols = img.shape[:2]
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(
    cameraMatrix=mtx,
    distCoeffs=dist,
    imageSize=(cols, rows),
    alpha=1,
    newImgSize=(cols, rows)
)
print(f'New camera matrix:\n{new_camera_mtx}')
print(f'Region of interest:\n{roi}')

In [ ]:
dst = cv2.undistort(
    src=img,
    cameraMatrix=mtx,
    distCoeffs=dist,
    dst=None,
    newCameraMatrix=new_camera_mtx
)

plt.imshow(dst)
plt.axis('off')
plt.show()

In [ ]:
roi_x, roi_y, roi_w, roi_h = roi
dst = dst[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]
plt.imshow(dst)
plt.axis('off')
plt.show()